### Importing Libraries


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import math


### Initialization of parameters

In [ ]:
mu_r = 2.0
epsilon_r = 6.0
mu_0 =1.25663706212E-6 
epsilon_0 =8.8541878128E-12
imp0 = np.sqrt(mu_0/epsilon_0)
delta_z = 0.004293 #converted from cm to m
N_z = 200
n_r = math.sqrt((mu_r*epsilon_r))
z = np.zeros((N_z,1))
#initialization of device parameters
Mu_r = np.ones((1,N_z))
Epsilon_r = np.ones((1,N_z))
print(Mu_r,Epsilon_r)
print(Mu_r.size,Epsilon_r.size)
#Adding the materials on the grid

Mu_r[:,13:83] = mu_r
Epsilon_r[:,13:83] = epsilon_r
print(Mu_r.shape,Epsilon_r.shape,z.shape)

### Initial computations of relevant parameters

In [ ]:
#Compute time step
c_0 =  299792458 #m/s
delta_t = (1*delta_z)/(2*c_0)
print(delta_t)

#Compute Source parameters
tau = 1/(2*1.5E9) #Width of gaussian pulse
t_0 = 4*tau     #offset in time for source
print(tau,t_0)

#Compute number of iterations
t_prop = (n_r*N_z*delta_z)/c_0
T = 12*tau + 5*t_prop
iteration = math.ceil(T/delta_t)
t = np.linspace(0,iteration*delta_t,iteration)
print(T,iteration,t.shape)

### Defining source excitation

In [ ]:
def gaussian_source(t,t_0,tau,delta_t,delta_z,c_0):
    #Set source permittivity and permeability
    mu_src =1 #these parameters should be the material permittivity/permeability at the grid position of the source injection
    epsilon_src = 1
    n_src = np.sqrt(epsilon_src*mu_src)
    delta = (n_src*delta_z/2*c_0)+(delta_t/2)
    A = -np.sqrt(epsilon_src/mu_src)
    x = (t - t_0)/tau
    Esrc = np.exp(-np.power(x,2))
    Hsrc = A*np.exp(-np.power(x,2))
    return Esrc,Hsrc

### Source excitation initialization

In [ ]:
Esrc,Hsrc = gaussian_source(t,t_0,tau,delta_t,delta_z,c_0)
print(z.shape,Esrc.shape,Hsrc.shape)
plt.figure(figsize=[20,5])
plt.plot(t,Esrc)
plt.plot(t,Hsrc)
injection_point = 100
print(max(Esrc),min(Esrc),max(Hsrc),min(Hsrc))

### Main FDTD Loop (Basic Algorithm)

In this part, there is no excitation so n

In [ ]:
#FDTD Loop
#Creating Field values
E = np.zeros((1,N_z))
H_norm = np.zeros((1,N_z))

#Update coefficient values
m_E = c_0*delta_t/1
m_H = c_0*delta_t/1
E.shape,H_norm.shape
plt.figure(1)
#Update Equations (Looping in time)
for i in range(iteration):
    #Looping in space
    for j in range(N_z-1):
        H_norm[:,j] = H_norm[:,j] + m_H*((E[:,j+1] - E[:,j])/delta_z)
    #Boundary condition
    H_norm[:,N_z-1] = H_norm[:,N_z-1] + m_H*(-E[:,N_z-1]/delta_z)
    
    #Boundary condition
    E[:,0] = E[:,j] + m_E*((H_norm[:,j]-H_norm[:,j-1])/delta_z)
    #looping in space (Electric Field)
    for j in range(1,N_z):
        E[:,j] = E[:,j] + m_E*(H_norm[:,j]/delta_z)
    
    
    
    plt.plot(E.T)#Plot E-Field
    plt.plot(H_norm.T)#Plot H-Field
    plt.ylabel('Value')
    plt.xlabel('z (Space)')
    plt.title(f'FDTD 1-D Iteration step: {i}/{iteration}')
    plt.show()
    #lineE.set_xdata(z)
    #lineE[0].set_ydata(E[i].T)
    #lineH.set_xdata(z)
    ##lineH[0].set_ydata(H_norm[i].T)
    #plt.pause(1)
    #fig.show()
    
print(E.shape,H_norm.shape)
        

#### FDTD Loop (Algorithm with Soft source) with Dirichlet Boundary Condition

In [ ]:

#FDTD Loop
#Creating Field values
E = np.zeros((1,N_z))
H_norm = np.zeros((1,N_z))
#Update coefficient values
m_E = c_0*delta_t/1
m_H = c_0*delta_t/1
E.shape,H_norm.shape

E_plot = np.zeros([iteration,N_z])
H_norm_plot = np.zeros([iteration,N_z])
print(E_plot.shape, H_norm_plot.shape)


#Update Equations (Looping in time)
for i in range(iteration):
    #Looping in space
    for j in range(N_z-1):
        H_norm[:,j] = H_norm[:,j] + m_H*((E[:,j+1] - E[:,j])/delta_z)
        if j == injection_point: #Injecting the source
            H_norm[:,j] = H_norm[:,j] + Esrc[i]
    #Boundary condition
    H_norm[:,N_z-1] = H_norm[:,N_z-1] + m_H*(-E[:,N_z-1]/delta_z)
    
    #Boundary condition
    E[:,0] = E[:,j] + m_E*(H_norm[:,0]/delta_z)
    #looping in space (Electric Field)
    for j in range(1,N_z):
        E[:,j] = E[:,j] + m_E*((H_norm[:,j] - H_norm[:,j-1])/delta_z)
        if j == injection_point: #Injecting the source
            E[:,j] = E[:,j] + Hsrc[i]
    
    E_plot[i,:] = E
    H_norm_plot[i,:] = H_norm
    print("=============================================================")
    print(f"Successfully computed field values! Iteration {i}/{iteration}....")

#### FDTD Hard source with Dirichlet Boundary Condition

In [ ]:

#FDTD Loop
#Creating Field values
E = np.zeros((1,N_z))
H_norm = np.zeros((1,N_z))
#Update coefficient values
m_E = c_0*delta_t/1
m_H = c_0*delta_t/1
E.shape,H_norm.shape

E_plot = np.zeros([iteration,N_z])
H_norm_plot = np.zeros([iteration,N_z])
print(E_plot.shape, H_norm_plot.shape)


#Update Equations (Looping in time)
for i in range(iteration):
    #Looping in space
    for j in range(N_z-1):
        H_norm[:,j] = H_norm[:,j] + m_H*((E[:,j+1] - E[:,j])/delta_z)
        if j == injection_point - 1: #Injecting the source
            H_norm[:,j] =  Esrc[i]
    #Boundary condition
    H_norm[:,N_z-1] = H_norm[:,N_z-1] + m_H*(-E[:,N_z-1]/delta_z)
    
    #Boundary condition
    E[:,0] = E[:,0] + m_E*(H_norm[:,0]/delta_z)
    #looping in space (Electric Field)
    for j in range(1,N_z):
        E[:,j] = E[:,j] + m_E*((H_norm[:,j] - H_norm[:,j-1])/delta_z)
        if j == injection_point: #Injecting the source
            E[:,j] =  Hsrc[i]
    
    E_plot[i,:] = E
    H_norm_plot[i,:] = H_norm
    print("=============================================================")
    print(f"Successfully computed field values! Iteration {i}/{iteration}....")

#### 1D FDTD Soft Source (with PBC)

In [ ]:

#FDTD Loop
#Creating Field values
E = np.zeros((1,N_z))
H_norm = np.zeros((1,N_z))
#Update coefficient values
m_E = c_0*delta_t/(1*delta_z)
m_H = c_0*delta_t/(1*delta_z)
E.shape,H_norm.shape

E_plot = np.zeros([iteration,N_z])
H_norm_plot = np.zeros([iteration,N_z])
print(E_plot.shape, H_norm_plot.shape)

#Initializing the perfect boundary condition..
e1 = 0
e2 = 0
e3 = 0 
h1 = 0
h2 = 0
h3 = 0

#Update Equations (Looping in time)
for i in range(iteration):
    
    #Looping in space
    for j in range(0,N_z-1):
        H_norm[:,j] = (H_norm[:,j] + m_H*((E[:,j+1] - E[:,j])))
                                          
    #Boundary condition
    H_norm[:,N_z-1] = (H_norm[:,N_z-1] + m_H*( e3-E[:,N_z-1]))
    h3=h2
    h2=h1
    h1=H_norm[:,0]
    
                                          
    #Boundary condition
    E[:,0] = E[:,0] + m_E*(H_norm[:,0]-h3)
    #looping in space (Electric Field)
    for j in range(1,N_z):
        E[:,j] = E[:,j] + m_E*((H_norm[:,j] - H_norm[:,j-1]))
    
    e3=e2
    e2=e1
    e1=E[:,N_z-1]
    
    #Source Injection (Soft Source)
    E[:,injection_point] = E[:,injection_point] + Esrc[i]
    

    
    E_plot[i,:] = E
    H_norm_plot[i,:] = H_norm
    print("=============================================================")
    print(f"Successfully computed field values! Iteration {i}/{iteration}....")

In [ ]:
%matplotlib notebook
fig = plt.figure(1,[10,5])
ax = fig.add_subplot(111)
plt.ion()

fig.show()
fig.canvas.draw()

for i in range(0,iteration):
    lineE = E_plot[i,:]
    #lineH = H_norm_plot[i,:]
    ax.clear()
    #plt.legend(handles = [lineE,lineH],labels=["Electric Field","Magnetic Field"])
    plt.ylabel('Value')
    plt.axvline(x=injection_point, color = "grey", linestyle="--")
    plt.ylim(-5,5)
    plt.xlabel('z (Space)')
    #plt.ylimit([-3,3])/
    plt.title(f'FDTD 1-D Iteration step: {i}/{iteration}')
    ax.plot(lineE)
    #ax.plot(lineH)
    fig.canvas.draw()
    #plt.savefig(f"1d-fdtd{i}.jpeg")
    ax = plt.axes()
    


In [4]:
import numpy as np
from math import exp
from matplotlib import pyplot as plt
ke = 200
ex = np.zeros(ke)
hy = np.zeros(ke)
# Pulse parameters
kc = int(ke / 2)
t0 = 40
spread = 12
boundary_low = [0, 0]
boundary_high = [0, 0]
nsteps = 250
# Dictionary to keep track of desired points for plotting
plotting_points = [{'num_steps': 100, 'data_to_plot': None, 'label': ''},
                   {'num_steps': 225, 'data_to_plot': None, 'label': ''},
                   {'num_steps': 250, 'data_to_plot': None, 'label': 'FDTD cells'}]
# Main FDTD Loop
for time_step in range(1, nsteps + 1):
    # Calculate the Ex field
    for k in range(1, ke):
        ex[k] = ex[k] + 0.5 * (hy[k - 1] - hy[k])
    # Put a Gaussian pulse in the middle
    pulse = exp(-0.5 * ((t0 - time_step) / spread) ** 2)
    ex[kc] = pulse
    # Absorbing Boundary Conditions
    ex[0] = boundary_low.pop(0)
    boundary_low.append(ex[1])
    ex[ke - 1] = boundary_high.pop(0)
    boundary_high.append(ex[ke - 2])
    # Calculate the Hy field
    for k in range(ke - 1):
        hy[k] = hy[k] + 0.5 * (ex[k] - ex[k + 1])
            
    # Save data at certain points for later plotting
    for plotting_point in plotting_points:
        if time_step == plotting_point['num_steps']:
            plotting_point['data_to_plot'] = np.copy(ex)
            
# Plot the outputs as shown in Fig. 1.3
plt.rcParams['font.size'] = 12
fig = plt.figure(figsize=(8, 5.25))
def plot_e_field(data, timestep, label):
    """Plot of E field at a single time step"""
    plt.plot(data, color='k', linewidth=1)
    plt.ylabel('E$_x$', fontsize='14')
    plt.xticks(np.arange(0, 199, step=20))
    plt.xlim(0, 199)
    plt.yticks(np.arange(0, 1.2, step=1))
    plt.ylim(-0.2, 1.2)
    plt.text(100, 0.5, 'T = {}'.format(timestep),horizontalalignment='center')
    plt.xlabel('{}'.format(label))
    # Plot the E field at each of the time steps saved earlier
    for subplot_num, plotting_point in enumerate(plotting_points):
        ax = fig.add_subplot(3, 1, subplot_num + 1)
        plot_e_field(plotting_point['data_to_plot'],
                     plotting_point['num_steps'],
                     plotting_point['label'])
    plt.tight_layout()
    #plt.show()
    

<Figure size 576x378 with 0 Axes>